In [19]:
!pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [38]:
!pip install numpy
!pip install pandas
!pip install multiprocessing
!pip install botocore
!pip install boto3

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\naoya\Anaconda3\envs\ml-agents\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\naoya\\AppData\\Local\\Temp\\pip-install-v553bbnt\\multiprocessing\\setup.py'"'"'; __file__='"'"'C:\\Users\\naoya\\AppData\\Local\\Temp\\pip-install-v553bbnt\\multiprocessing\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\naoya\AppData\Local\Temp\pip-pip-egg-info-ykipplee'
         cwd: C:\Users\naoya\AppData\Local\Temp\pip-install-v553bbnt\multiprocessing\
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1

In [40]:
from urllib import request
def http_download(url, path):
    with request.urlopen(url) as f:
        with open(path, "wb") as fout:
            buf = f.read(1024)
            while buf:
                fout.write(buf)
                buf = f.read(1024)

from multiprocessing import Pool, Manager
import functools
def batch_download(bucket, file_paths, root, num_workers=10, retry=10):
    with Pool(num_workers) as p:
        m = Manager()
        counter = m.Value('i', 0)
        lock = m.Lock()
        download_ = functools.partial(download, bucket, root, retry, counter, lock)
        p.map(download_, file_paths)

def log_counts(values):
    for k, count in values.value_counts().iteritems():
        logging.warning(f"{k}: {count}/{len(values)} = {count/len(values):.2f}.")
        
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import botocore

def download(bucket, root, retry, counter, lock, path):
    i = 0
    src = path
    dest = f"{root}/{path}"
    while i < retry:
        try:
            if not os.path.exists(dest):
                s3.download_file(bucket, src, dest)
            else:
                logging.info(f"{dest} already exists.")
            with lock:
                counter.value += 1
                if counter.value % 100 == 0:
                    logging.warning(f"Downloaded {counter.value} images.")
            return
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                logging.warning(f"The file s3://{bucket}/{src} does not exist.")
                return
            i += 1
            logging.warning(f"Sleep {i} and try again.")
            time.sleep(i)
    logging.warning(f"Failed to download the file s3://{bucket}/{src}. Exception: {e}")

In [37]:
#download open-images, make my own dataset!!!!

bucket = "open-images-dataset"
import pandas as pd
import logging
import os
class_description_file = './class-descriptions-boxable.csv' #bofore download CLI 
class_descriptions = pd.read_csv(class_description_file,names=["id", "ClassName"])
#learn bottle data
class_names = ["Bottle"]
group_filters = [""]
percentages = [1.0]
excluded_images = set()
class_descriptions = class_descriptions[class_descriptions['ClassName'].isin(class_names)]

image_files = []
object_dir = "./images"
os.makedirs(object_dir, exist_ok=True)
for dataset_type in ["train", "validation", "test"]:
    image_dir = os.path.join(object_dir, dataset_type)

    #csvのダウンロード(バカ長い)
    os.makedirs(image_dir, exist_ok=True)
    annotation_file = f"{image_dir}/{dataset_type}-annotations-bbox.csv"
    if not os.path.exists(annotation_file):
        url = f"https://storage.googleapis.com/openimages/2018_04/{dataset_type}/{dataset_type}-annotations-bbox.csv"
        logging.warning(f"Download {url}.")
        http_download(url, annotation_file)

    logging.warning(f"Read annotation file {annotation_file}")
    annotations = pd.read_csv(annotation_file)
    annotations = pd.merge(annotations, class_descriptions,left_on="LabelName", right_on="id",how="inner")
    annotations = annotations.loc[annotations['IsDepiction'] != 1, :] #?

    filtered = []
    for class_name, group_filter, percentage in zip(class_names, group_filters, percentages):
        sub = annotations.loc[annotations['ClassName'] == class_name, :]
        excluded_images |= set(sub['ImageID'].sample(frac=1 - percentage))

        if group_filter == '~group':
            excluded_images |= set(sub.loc[sub['IsGroupOf'] == 1, 'ImageID'])
        elif group_filter == 'group':
            excluded_images |= set(sub.loc[sub['IsGroupOf'] == 0, 'ImageID'])
        filtered.append(sub)

    annotations = pd.concat(filtered)
    annotations = annotations.loc[~annotations['ImageID'].isin(excluded_images), :]

    annotations = annotations.sample(frac=1.0)

    logging.warning(f"{dataset_type} bounding boxes size: {annotations.shape[0]}")
    logging.warning("Approximate Image Stats: ")
    log_counts(annotations.drop_duplicates(["ImageID", "ClassName"])["ClassName"])
    logging.warning("Label distribution: ")
    log_counts(annotations['ClassName'])

    logging.warning(f"Shuffle dataset.")


    sub_annotation_file = f"{object_dir}/sub-{dataset_type}-annotations-bbox.csv"
    logging.warning(f"Save {dataset_type} data to {sub_annotation_file}.")
    annotations.to_csv(sub_annotation_file, index=False)
    image_files.extend(f"{dataset_type}/{id}.jpg" for id in set(annotations['ImageID']))
logging.warning(f"Start downloading {len(image_files)} images.")
batch_download(bucket, image_files, object_dir, 10, 10)
logging.warning("Task Done.")

NameError: name 'download' is not defined

In [20]:
#確認
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

True
NVIDIA GeForce GTX 960


In [15]:
#make model
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_small_ssd_lite
from vision.ssd.config import mobilenetv1_ssd_config
create_net = lambda num: create_mobilenetv3_small_ssd_lite(num)
config = mobilenetv1_ssd_config

In [16]:
#prepare tranform
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform
from vision.ssd.ssd import MatchPrior
train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform = TestTransform(config.image_size, config.image_mean, config.image_std)
logging.info("Prepare training datasets.")

In [17]:
#TODO データセットのライブラリ作る、transformも
from vision.datasets.open_images import OpenImagesDataset
datasets = []
dataset = OpenImagesDataset(dataset_path,transform=train_transform, target_transform=target_transform, dataset_type="train", balance_data=args.balance_data)
label_file = os.path.join(args.checkpoint_folder, "open-images-model-labels.txt")
store_labels(label_file, dataset.class_names)
logging.info(dataset)
num_classes = len(dataset.class_names)

dataset_path = object_dir

dataset = OpenImagesDataset(dataset_path,transform=train_transform, target_transform=target_transform, dataset_type="train", balance_data=args.balance_data)
label_file = os.path.join(args.checkpoint_folder, "open-images-model-labels.txt")
store_labels(label_file, dataset.class_names)
logging.info(dataset)
num_classes = len(dataset.class_names)
logging.info(f"Stored labels into file {label_file}.")
train_dataset = ConcatDataset(datasets)
logging.info("Train dataset size: {}".format(len(train_dataset)))
train_loader = DataLoader(train_dataset, args.batch_size,num_workers=args.num_workers,shuffle=True)

val_dataset = OpenImagesDataset(dataset_path,transform=test_transform, target_transform=target_transform, dataset_type="test")
logging.info(val_dataset)
logging.info("validation dataset size: {}".format(len(val_dataset)))
val_loader = DataLoader(val_dataset, args.batch_size,num_workers=args.num_workers,shuffle=False)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
Param = {
    "batch_size" : 5,
    "base_net_lr" : 0.001,
    "num_epochs" : 100,
    "validation_epochs" : 5,
    "t_max" : 100,
    "lr" : 0.01,
    "scheduler" : "cosine",
    "freeze_base_net" : False #ファインチューニングするときはTrue,
    "freeze_net" : False
}

In [ ]:
import itertools

logging.info("Build network.")
net = create_net(num_classes)
min_loss = -10000.0
last_epoch = -1
base_net_lr = Param["base_net_lr"]
extra_layers_lr = Param["lr"]
if Param["freeze_base_net"] == True:
    logging.info("Freeze base net.")
    freeze_net_layers(net.base_net)
    params = itertools.chain(net.source_layer_add_ons.parameters(), net.extras.parameters(),
                            net.regression_headers.parameters(), net.classification_headers.parameters())
    params = [
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]
elif Param["freeze_base_net"] == True:
    freeze_net_layers(net.base_net)
    freeze_net_layers(net.source_layer_add_ons)
    freeze_net_layers(net.extras)
    params = itertools.chain(net.regression_headers.parameters(), net.classification_headers.parameters())
    logging.info("Freeze all the layers except prediction heads.")
else:
    params = [
        {'params': net.base_net.parameters(), 'lr': base_net_lr},
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]

In [ ]:
#next モデル定義 -> 初めからやるぞ！！！

In [ ]:
from vision.nn.multibox_loss import MultiboxLoss

criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                             center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=args.lr, momentum=args.momentum,
                            weight_decay=args.weight_decay)

In [ ]:
def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)  # TODO CHANGE BOXES
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        if i and i % debug_steps == 0:
            avg_loss = running_loss / debug_steps
            avg_reg_loss = running_regression_loss / debug_steps
            avg_clf_loss = running_classification_loss / debug_steps
            logging.info(
                f"Epoch: {epoch}, Step: {i}, " +
                f"Average Loss: {avg_loss:.4f}, " +
                f"Average Regression Loss {avg_reg_loss:.4f}, " +
                f"Average Classification Loss: {avg_clf_loss:.4f}"
            )
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0

In [ ]:
def test(loader, net, criterion, device):
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num